# Luftüberwachung - Analyse der Ergebnisse

In dem folgendem Jupiter Notebook werden die Daten aus dem AWS Datenspeicher aufbereitet und visualsiert. Basierend auf der Analyse werdem den Endverbrauche die wichtigsten Erkenntnisse per Telegram Bot auf das Handy geschickt. 

Da die Auswertung über den Telegram-Bot erfolgen soll, werden die verschiedenen Schritte hier als Funktion definiert. Am Ende wird der Telegram-Bot definiert und gestartet.

Für einen reibungslosen Ablauf sollte, das Notebook einmalig nach dem Hochfahren am Stück ausgeführt werden.

### Vorbereitenende Maßnahmen

In [1]:
%matplotlib inline

In [2]:
import boto3
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
import requests
from telegram.ext import *

In [3]:
client = boto3.client("iotanalytics")

## Import  und Filtern der Daten

### Konvertieren der Zeitzone von UTC zu Europe/Berlin

In [4]:
def convert_timezone(df):
    df.index = df.index.tz_localize('UTC').tz_convert('Europe/Berlin').tz_localize(None)
    return df

### Import der Daten und Aufbereitung

In [5]:
def import_data(setName): # Daten werden importiert, der Timestamp in einen Datetype geändert und als Index gesetzt
    data = client.get_dataset_content(datasetName=setName)
    df = pd.read_csv(data["entries"][0]["dataURI"])
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df.drop("__dt", axis = 1, inplace = True)
    df.set_index("timestamp", inplace = True)
    df=df.sort_index()
    df = convert_timezone(df)
    return df

### Löschen aller Werte über 4000 PPM

In [6]:
def remove_errors(df):
    df.drop(df.index[df['co2'] >= 4000], inplace = True)
    return df

### Lücken in den Daten mit None-Werten füllen (relevant für Darstellung)

In [7]:
def fill_df(df):
    new_row = pd.Series({'co2':None, 'humidity':None, 'temperature':None})
    for i in range(len(df)-1):
        # Check if timedelta between timestamps is bigger than 30 secs and value of timestamp is not None, if so add None values
        delta = (df.iloc[i+1].name - df.iloc[i].name).seconds
        if delta > 30 and df.iloc[i]['co2'] != None:
            new_row.name = df.iloc[i].name + timedelta(seconds = 6)
            df = df.append(new_row, ignore_index=False)
    return df

In [8]:
def filter_last_days(df,days = 7): 
    # Filter the last x days, default = 7
    today = date.today()
    week_ago = today - timedelta(days=days)
    tomorow = today + timedelta(days=1)
    df_filtered = df.loc[week_ago:tomorow]
    return df_filtered
    

In [9]:
def filter_last_days_2(df,start = 0,days = 7): 
    # Filter the last start+days days beginning with start, default: start=0, days=7
    today = date.today()
    week_ago = today - timedelta(days=(days+start))
    tomorow = today - timedelta(days=start)
    df_filtered = df.loc[week_ago:tomorow]
    return df_filtered

### Berechnung Grenzwertüberschreitung Co2

In [10]:
def bounder_check(df, lower_bound= None, upper_bound = None): 
    # Returns percentage of Values in bounder(s)
    # Bounders are included
    if lower_bound == None:
        out_bounder = df[df >= upper_bound]
    elif upper_bound == None:
        out_bounder = df[df <= lower_bound]
    else:
        out_bounder = df[(df <= lower_bound)] & df[(df >= upper_bound)]
    
    percent_out_bounder = len(out_bounder.index) / len(df.index) * 100
    bounder_check_result = round(100 - percent_out_bounder,2)
    return bounder_check_result

In [11]:
def complete_bounder_check(df):
    # Bounder check for complete df
    bounder_check_result_co2 = bounder_check(df['co2'],upper_bound=800)
    bounder_check_result_temp = bounder_check(df['temperature'],20,24)
    bounder_check_result_humi = bounder_check(df['humidity'],40,60)
    return [bounder_check_result_co2,bounder_check_result_temp,bounder_check_result_humi]

In [12]:
def compare(df_old,df,bounder_check_old,bounder_check,days):
    
    #df_old_described = df_old.describe()
    #df_described = df.describe()
    #df_compared = df_described.describe()/df_old_described.describe()-1
    bounder_check_compare= []
    for i in range(len(bounder_check_old)):
        bounder_check_compare.append(round(bounder_check[i]-bounder_check_old[i],2))
    message = ('In den letzten '+ str(days) + ' Tagen, hat sich dein Fortschritt im Vergleich zu den vorherigen ' + str(days) +
               ' Tagen um folgende Prozentpunkte verändert:\n'
        'CO2 '+ str(bounder_check_compare[0])+ ' %P\n'
        'Temperatur '+ str(bounder_check_compare[1]) + ' %P\n'
        'Luftfeutigkeit '+ str(bounder_check_compare[2]) + ' %P')
    return message

In [13]:
def bounder_cross(df,lower_bounder = None, upper_bounder = None):
    counter = 0
    if lower_bounder == None:
        for i in range(1,len(df)):
            if (df[i] >= upper_bounder and df[i-1] < upper_bounder):
                counter +=1
    elif upper_bounder == None:
        for i in range(1,len(df)):
            if (df[i] < lower_bounder and df[i-1] >= lower_bounder):
                counter +=1
    else:
        for i in range(1,len(df)):
            if (df[i] >= upper_bounder and df[i-1] < upper_bounder) or (df[i] <= lower_bounder and df[i-1] > lower_bounder):
                counter +=1
    return counter

In [14]:
def compare_bounder_cross(df,df_old,lower_bounder = None, upper_bounder = None):
    df_old_bounder_cross = bounder_cross(df_old,lower_bounder, upper_bounder)
    df_bounder_cross = bounder_cross(df,lower_bounder, upper_bounder)
    return round((df_bounder_cross/df_old_bounder_cross-1)*100,2)

## Visualisierung der Daten  

Der Anwender erhählt die Auswertung der Messdaten in verschiedenen Formen. Zum einen bekommt er eine Gesamtübersicht über die vergangenen Woche. Da diese zum Teil sehr unübersichtlich ist, bekommt er er zusätlich eine einfach aufgearbeitet Ansicht in Ringform. Angelehnt an die "Aktivitätsringe" der Apple Watch soll der Anwender ein Gefühl dafür bekommen, wie es um seine Luftumgebung steht. Sind alle Ringe geschlossen, hatte der Anwender in der vergangenen Woche eine optimale Luftumgebung. 

### Verlauf

In [15]:
def graph(df):
    ax = plt.gca()
    ax1 = df.plot(use_index=True, y="co2",  figsize=(20, 10),ylim=(0,2500), ax=ax)
    ax2 = df.plot(use_index=True, y=["humidity","temperature"], secondary_y=True, ax=ax, grid = True)
    file_path = 'sensor_data.png'

    #design Bericht
    ax.set_xlabel("Zeit",size = 15)
    ax.set_ylabel('PPM',size = 15)
    ax2.set_ylabel('% / °C', size = 15)
    ax.set_facecolor('white')
    ax.set_title("Verlauf", size = 30, color = "black")
    plt.savefig(file_path)
    plt.close()

### Ringübersicht

In [16]:
def donut(data):
    scenario_all = ['1. Co2-Gehalt','2. Luftfeuchtigkeit','3. Temperatur']
    percentage_all = [ str(data[0]) , str(data[2]), str(data[1])]
    num_all_1 = ['1.','2.','3.']

    color = ['#5cdb6f','#6f5cdb','#db6f5c']

    # create donut plots
    startingRadius = 0.7 + (0.3* (len(scenario_all)-1))
    fig = plt.figure()
    for i in range(3):
        scenario = scenario_all[i]
        num_all = num_all_1[i]
        percentage = percentage_all[i]
        textLabel = str(scenario) + '\n ' + str(percentage) + '%'
        textLabel_1 = str(num_all) +' ' + str(percentage) + '%'

        remainingPie = 100 - float(percentage)

        donut_sizes = [remainingPie, percentage]
        
        plt.pie(donut_sizes, radius=startingRadius, startangle=90, colors=['#ffffff', color[i]],
                wedgeprops={"edgecolor": "grey", 'linewidth': 1})
        plt.text(1.3, startingRadius - 0.10, textLabel, horizontalalignment='left', verticalalignment='center', size = 10)
    
        plt.text(0.0, startingRadius - 0.20, textLabel_1, horizontalalignment='left', verticalalignment='center')
    
        
    
        plt.title("Dein Luftüberwachungsfortschritt", size = 15)

        startingRadius-=0.3

    # equal ensures pie chart is drawn as a circle (equal aspect ratio)
    plt.axis('equal')

    # create circle and place onto pie chart
    circle = plt.Circle(xy=(0, 0), radius=0.35, facecolor='grey')
    plt.gca().add_artist(circle)

    plt.savefig('piePlot.png',dpi=fig.dpi)
    plt.close()


## Telegram-Bot

### Funktionen für einzelne Befehle

In [17]:
def analyse(days):
    # Import Data
    df_air = import_data('room_air_dataset')
    # Filter last days
    df_air_filt = filter_last_days(df_air,days)
    df_air_filt_old = filter_last_days_2(df_air,days,days*2)
    # Remove values when to high
    df_air_filt = remove_errors(df_air_filt)
    df_air_filt_old = remove_errors(df_air_filt_old)
    # Calcuate Data for Circles
    circles = complete_bounder_check(df_air_filt)
    circles_old = complete_bounder_check(df_air_filt_old)
    # Create Circles
    donut(circles)
    # Prepare Data for graph
    df_graph_data = fill_df(df_air_filt)
    # Create Graph
    graph(df_graph_data)
    # Comparsion to delta -1
    message = compare(df_air_filt_old,df_air_filt,circles_old,circles,days)
    result_bounder_cross = compare_bounder_cross(df_air_filt['co2'],df_air_filt_old['co2'], upper_bounder=800)
    if result_bounder_cross >= 0:
        change = 'zugenommen.'
    else:
        change = 'abgenommen.'
    
    message = message + '\nDie Grenzüberschreitungen beim Co2 haben um ' + str(abs(result_bounder_cross)) + ' % ' + change
    return message

In [18]:
def live():
    df_air = import_data('room_air_dataset')
    live_data = df_air.iloc[-1]
    message = ('Die letzten gemessenen Werte:\n'
               'Uhrzeit '+ str(live_data.name)+ ' \n'
               'CO2             '+ str(live_data['co2'])+ ' PPM\n'
               'Temperatur      '+ str(live_data['temperature']) + ' °C\n'
               'Luftfeutigkeit  '+ str(live_data['humidity']) + ' %')
    return message
    

### Telegram-Bot Funktionen

In [19]:
HEADERS = {
    'Accept': 'application/json'}
telegram_token = '5334156117:AAFoRuNdxE2CyXgLnfuWlQY1Tbm5qzQFuo4'
allowed_ids = [5342271588] #280530619
api_url = 'https://api.telegram.org/bot'+telegram_token+'/'


def handle_message(update, context):
    #print(update)
    text = update.message.text

    response = create_responses(text, update['message']['chat']['id'])
    update.message.reply_text(response)

    
def main():
    updater = Updater(telegram_token)
    dp = updater.dispatcher

    dp.add_handler(MessageHandler(Filters.text, handle_message))

    updater.start_polling(0)
    
    
def create_responses(input_text, id_chat):
    # Check if user is allowed
    input_text = input_text.lower()
    if id_chat in allowed_ids:
        if input_text == '/start':
            return ('Hallo, herzlich Willkommen bei deinem persönlichen Luftüberwachungsassistenten!' 
                    'Ich freue auf die gemeinsame Zusammenarbeit')
        
        elif input_text == 'hilfe':
            return ('Hast du Probleme mit dem Mikrocontroller, drücke die Resettaste und schaue ob das Problem behoben wurde.\n'
                    '\n'
                    'Folgende Befehle stehen Dir bei mir zur Verfügung: \n'
                    'Bericht (Anzahl)\n'
                    'Danke \n'
                    'Live \n'
                    'Kontakt')
        
        elif input_text == 'kontakt':
            return 'Unter der folgenden Nummer kannst du uns 24/7 erreichen: +49 123 456789'
        
        elif input_text == 'danke':
            return 'Gerne! Für weitere Anfragen stehe ich gerne zur Verfügung'
        
        elif input_text == 'live':
            message = live()
            return message
        
        elif 'bericht' in input_text:
            text = input_text.split()
            if len(input_text.split()) == 2:
                try:
                    number = int(text[1])
                except:
                    number = 7
            else: 
                number = 7
        
            telegram_send('Deine Auswertung über ' + str(number)+ ' Tage wird erstellt...',id_chat)
            try:
                message = analyse(number)
                send_photo(id_chat, open('piePlot.png', 'rb'))
                send_photo(id_chat, open('sensor_data.png', 'rb'))
                #sensor_data.png
                return message
            except Exception as e:
                print(str(e))
                return str(e)
        else:
            return 'Das habe ich nicht verstanden. Schicke bitte "Hilfe" oder "Kontakt", damit wir dir helfen können'
    else:
        return ('Du scheinst leider nicht die Berechtigung zu haben, mit dem Bot zu kommunzieren, ' 
                'bitte wende dich an den Botbesitzer')


def telegram_send(message: str,telegram_chat_id: int ) -> None:
    url = 'https://api.telegram.org/bot'+telegram_token+'/sendMessage'
    params = {
        'chat_id': telegram_chat_id,
        'parse_mode': 'Markdown',
        'text': message
    }
    r = requests.get(url, params=params, headers=HEADERS)

    
def send_photo(id_chat, file_opened):
    method = "sendPhoto"
    params = {'chat_id': id_chat}
    files = {'photo': file_opened}
    resp = requests.post(api_url + method, params, files=files)
    return resp

### Start des Telegram-Bots

In [20]:
if __name__ == "__main__":
    main()